# Section 2: Primary Analyses
## Sensor Space Analysis
### Prepare time domain epochs.

In [ ]:
import os
import numpy as np
from mne import read_epochs
from mne.filter import low_pass_filter
from pandas import read_csv, concat

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Subject level parameters.
subjects = ['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
task = 'msit'
event_id = ['FN', 'FI', 'NN', 'NI']
analysis = 'resp'
h_freq = 50

## Channel definitions. 
pick_channels = ['FZ']

## Processing parameters.
fmax = 15
decim = 3

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Iteratively load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

for n, subject in enumerate(subjects):
    
    ## Load behavioral data.
    csv = read_csv(os.path.join('ave','%s_%s_%s-epo.csv' %(subject,task,h_freq)))

    ## Load in epochs.
    epo_file = os.path.join('ave','%s_%s_%s_%s-epo.fif' %(subject,task,h_freq,analysis))
    epochs = read_epochs(epo_file, verbose=False)
    
    ## Restrict to times/channels of interest.
    if analysis == 'stim': epochs.crop(-0.5,2.0)
    elif analysis == 'resp': epochs.crop(-1.0,1.0)
    epochs.pick_channels(pick_channels)

    ## Extract epochs.
    Fs = epochs.info['sfreq']
    times = epochs.times
    epochs = epochs.get_data().swapaxes(0,1)
    
    ## Merge.
    if not n: 
        trials = epochs
        df = csv
    else:
        trials = np.concatenate([trials,epochs], axis=1)
        df = concat([df,csv])
        
## Apply lowpass filter. 
trials = low_pass_filter(trials, Fs, fmax, filter_length='20s', n_jobs=3, verbose=False)
df = df.reset_index(drop=True)

## Downsample.
trials = trials[:,:,::decim]
times = times[::decim]

## Convert to uV.
trials *= 1e6

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Make directories (if not already made). Save.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Make directory.
out_dir = 'sensor'
if not os.path.isdir(out_dir): os.makedirs(out_dir)
    
## Save data.
for arr, ch in zip(trials,pick_channels): np.savez_compressed(os.path.join(out_dir, 'afMSIT_sensor_%s_%s_%s' %(analysis,ch,fmax)),
                                                              data=arr, times=times, fmax=fmax)
df.to_csv(os.path.join(out_dir, 'afMSIT_sensor_info.csv'), index=False)

print 'Done.'

### Prepare power amplitudes for single-trial epochs.

In [ ]:
import os
import numpy as np
from mne import read_epochs
from mne.filter import low_pass_filter
from mne.time_frequency import single_trial_power
from pandas import read_csv

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Subject level parameters.
subjects = ['BRTU', 'CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
task = 'msit'
event_id = ['FN', 'FI', 'NN', 'NI']
h_freq = 50

## Channel definitions. 
pick_channels = ['FZ']

## Time-frequency parameters.
fdict = dict(theta = (4,8), alpha = (8,15), beta = (15,30))
freqs = np.logspace( np.log10(2), np.log10(50), num=25)
n_cycles = 3
baseline = (-0.5, -0.1)
Fs = 1450.
decim = 14
n_jobs = 3

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

epo_dir = 'ave'
root_dir = 'sensor'
df = read_csv(os.path.join(root_dir, 'afMSIT_sensor_info.csv'))

for ch in pick_channels:

    for analysis in ['stim', 'resp']:

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Iteratively load and merge.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        trials = []
        for subject in subjects:

            ## Load in epochs.
            epo_file = os.path.join(epo_dir, '%s_%s_%s_%s-epo.fif' %(subject,task,h_freq,analysis))
            epochs = read_epochs(epo_file, verbose=False)

            ## Get label index and extract.
            epochs.pick_channels([ch])
            trials.append(epochs._data.squeeze())

        ## Concatenate.
        trials = np.concatenate(trials, axis=0)
        times = epochs.times

        if not df.shape[0] == trials.shape[0]: raise ValueError('Incompatible dimensions!')
        
         #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Phase-lock removal.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
       
        for subject in df.Subject.unique():
            for dbs in [0, 1]:
                for cond in [0, 1]:
                    ix, = np.where((df.Subject==subject)&(df.DBS==dbs)&(df.Interference==cond))
                    trials[ix] -= trials[ix].mean(axis=0)
                    
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Compute power.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
                    
        print 'Computing single trial power: %s %s.' %(ch, analysis)
        trials = np.expand_dims(trials,1)
        power = single_trial_power(trials, sfreq=Fs, frequencies=freqs, n_cycles=n_cycles, 
                                    baseline=None, n_jobs=n_jobs, verbose=False)
        power = power.squeeze()
        del trials
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Within-trial normalization.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        power = (power.T / np.median(power, axis=-1).T).T # median, not mean
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Prepare baseline normalization (within subject, DBS).
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
        ## Compute baseline normalization.
        if analysis == 'stim':

            ## Make time mask.
            mask = (times >= baseline[0]) & (times <= baseline[1])
            
            ## Iteratively compute over subjects.
            blnorm = []
            for subject in df.Subject.unique():
                
                ## Iteratively compute over DBS conditions..
                sbl = []
                for dbs in [0,1]:

                    ix, = np.where((df.Subject==subject)&(df.DBS==dbs))
                    sbl.append( np.apply_over_axes(np.median, power[ix][:,:,mask], axes=[0,2]).squeeze() )
            
                blnorm.append(sbl)
            
            ## Merge.
            blnorm = np.array(blnorm)
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Apply baseline normalization (within subject, DBS).
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
        ## Setup index vectors.
        n_trials, n_freqs, n_times = power.shape        
        _, subj_ix = np.unique(df.Subject, return_inverse=True)
        dbs_ix = df.DBS.as_matrix()
        trial_ix = np.arange(n_trials) 
        
        ## Main loop.
        for i,j,k in zip(trial_ix,subj_ix,dbs_ix):
            
            for m in xrange(n_times):
                
                power[i,:,m] /= blnorm[j,k,:]
                
        if analysis == 'resp': del blnorm

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Final preprocessing steps.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
        power = np.log10(power) * 10
            
        ## Crop times.
        if analysis == 'stim': mask = (times >= -0.5) & (times <= 2.0)
        elif analysis == 'resp': mask = (times >= -1.0) & (times <= 1.0)
        power = power[:, :, mask]
        times = times[mask]
            
        ## Iteratively average, transform, and save.
        for k,v in fdict.iteritems():
            
            ## Average across frequencies.
            ix, = np.where((freqs>=v[0])&(freqs<=v[1]))
            band = power[:,ix,:].mean(axis=1)
        
            ## Save.
            f = os.path.join(root_dir, 'afMSIT_sensor_%s_%s_%s' %(analysis, ch, k))
            np.savez_compressed(f, data=band[:,::decim], times=times[::decim], freqs=freqs, n_cycles=n_cycles)
        
        del power
        
print 'Done.'

### Perform permutatations
Please see: scripts/cmdline/afMSIT_permutations.py

### Identify clusters, perform FDR corrections, and assemble results.

In [ ]:
import os
import numpy as np
from collections import defaultdict
from mne.stats import fdr_correction
from pandas import DataFrame, concat
from scipy.ndimage import measurements
from scipy.stats import norm

def largest_cluster(arr, threshold):
    masked = np.abs( arr ) > threshold
    clusters, n_clusters = measurements.label(masked)
    cluster_sums = measurements.sum(arr, clusters, index=np.arange(n_clusters)+1)
    if not len(cluster_sums): return 0
    else: return np.abs(cluster_sums).max()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Specify parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## File parameters.
labels = ['FZ', 'FCZ']
analysis = 'resp'
model_name = 'revised'
freqs = ['theta','alpha','beta']
domain = ['timedomain', 'frequency'][1]

## Statistics parameters.
alpha = 0.05
min_cluster = 0.05 # seconds

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Initial preparations.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define contrasts.
if model_name == 'revised': cols = ['Intercept', 'DBS', 'Interference', 'DBSxInt', 'nsArousal', 'nsValence', 'Trial']

## Define threshold.
threshold = norm.ppf(1 - alpha/2.)

## Read in seeds.
f = 'scripts/cmdline/seeds.txt'
with open(f, 'r') as f: seeds = [s.strip() for s in f.readlines()]
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main Loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#    

df = []
    
for label in labels:
    
    for freq in freqs:
    
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Load files. Assemble permutations.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        root_dir = 'sensor'
        results_dir = os.path.join(root_dir, model_name)
        out_dir = os.path.join(root_dir, 'results')

        ## Load true statistics.
        npz = np.load(os.path.join(results_dir, 'afMSIT_sensor_%s_%s_%s_obs.npz' %(analysis, label, freq)))
        t_scores = npz['t_scores'].squeeze()
        times = npz['times'].squeeze()

        ## Load null statistics.
        for n, seed in enumerate(seeds):
            npz = np.load(os.path.join(results_dir, 'afMSIT_sensor_%s_%s_%s_%s.npz' %(analysis, label, freq, seed)))
            if not n: permuted = npz['t_scores']
            else: permuted = np.concatenate([permuted, npz['t_scores']], axis=0)
                
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Compute cluster statistics.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        ## Get info.
        n_shuffles, n_eff, n_times  = permuted.shape

        ## Iteratively compute clusters.
        results = defaultdict(list)

        for n, con in enumerate(cols):

            ## Find real clusters.
            masked = np.abs( t_scores[n] ) > threshold
            clusters, n_clusters = measurements.label(masked)
            cluster_sums = measurements.sum(t_scores[n], clusters, index=np.arange(n_clusters)+1)

            ## Compute null cluster sums.
            null_sums = np.array([largest_cluster(permuted[m, n, :], threshold) for m in xrange(n_shuffles)])

            ## Compute cluster bounds.
            tmin = np.array([times[clusters==i].min() for i in np.arange(n_clusters)+1])
            tmax = np.array([times[clusters==i].max() for i in np.arange(n_clusters)+1])

            ## Find proportion of clusters that are larger.
            p_values = [(np.abs(cs) < null_sums).mean() for cs in cluster_sums]

            ## Store results.
            for t1, t2, cs, pval in zip(tmin,tmax,cluster_sums,p_values):
                results['Freq'] += [freq]
                results['Label'] += [label]
                results['Contrast'] += [con]
                results['Tmin'] += [t1]
                results['Tmax'] += [t2]
                results['Score'] += [cs]
                results['Pval'] += [pval]

        ## Organize results and append.
        results = DataFrame(results)
        results['Tdiff'] = results['Tmax'] - results['Tmin']
        results = results[results['Tdiff']>=min_cluster]
        df.append(results)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Compute cluster statistics.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Merge dataframes.
df = concat(df)

## Organize columns and sort.
cols = ['Contrast','Label','Freq','Tmin','Tmax','Tdiff','Score','Pval']
df = df[cols].sort_values(['Contrast','Tmin']).reset_index(drop=True)

## FDR correct within contrasts.
df['FDR'] = 0
for contrast in df.Contrast.unique():
    _, fdr = fdr_correction(df.loc[df.Contrast==contrast,'Pval'], alpha)
    df.loc[df.Contrast==contrast,'FDR'] = fdr
    
## Save.
f = os.path.join(out_dir, '%s_%s_%s_results.csv' %(model_name, analysis,domain))
df.to_csv(f, index=False)
        
print 'Done.'

## Source Space Analysis
### Source localize single trial epochs

In [ ]:
import os
import numpy as np
import pylab as plt
from mne import read_epochs, read_label, read_source_spaces, set_log_level
from mne.minimum_norm import apply_inverse_epochs, read_inverse_operator
from scipy.io import loadmat
set_log_level(verbose=False)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Subject level parameters.
subjects = ['BRTU','CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
task = 'msit'
analysis = 'resp'
parc = 'april2016'
fmax = 50

## Source localization parameters.
method = 'dSPM'
snr = 1.0  
lambda2 = 1.0 / snr ** 2
pick_ori = 'normal'

## Labels
rois = ['dacc-lh', 'dacc-rh', 'dmpfc-lh', 'dmpfc-rh', 'dlpfc_1-lh', 'dlpfc_1-rh', 'dlpfc_2-lh', 'dlpfc_2-rh', 
        'dlpfc_3-lh', 'dlpfc_3-rh', 'dlpfc_4-lh', 'dlpfc_4-rh', 'dlpfc_5-lh', 'dlpfc_5-rh', 
        'dlpfc_6-lh', 'dlpfc_6-rh', 'pcc-lh', 'pcc-rh', 'racc-lh', 'racc-rh']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Iteratively load and prepare data.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

root_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/afMSIT'
fs_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/freesurfs'

## Prepare fsaverage source space.
src = read_source_spaces(os.path.join(fs_dir,'fscopy','bem','fscopy-oct-6p-src.fif'))
vertices_to = [src[n]['vertno'] for n in xrange(2)]
labels = [read_label(os.path.join(fs_dir,'fscopy','label','april2016','%s.label' %roi), subject='fsaverage')
          for roi in rois]

for subject in subjects:

    print 'Performing source localization: %s' %subject

    ## Load in epochs.
    epochs = read_epochs(os.path.join(root_dir,'ave','%s_%s_%s_%s-epo.fif' %(subject,task,fmax,analysis)))
    times = epochs.times
    
    ## Load in secondary files.
    inv = read_inverse_operator(os.path.join(root_dir,'cov','%s_%s_%s-inv.fif' %(subject,task,fmax)))
    morph_mat = loadmat(os.path.join(root_dir, 'morph_maps', '%s-fsaverage_morph.mat' %subject))['morph_mat']

    ## Make generator object.
    G = apply_inverse_epochs(epochs, inv, method=method, lambda2=lambda2, pick_ori=pick_ori, return_generator=True)
    del epochs, inv

    ## Iteratively compute and store label timecourse. 
    ltcs = []
    for g in G:
        g = g.morph_precomputed('fsaverage', vertices_to=vertices_to, morph_mat=morph_mat)
        ltcs.append( g.extract_label_time_course(labels, src, mode='pca_flip') )
    ltcs = np.array(ltcs)
    
    ## Save.
    f = os.path.join(root_dir,'source','stcs','%s_%s_%s_%s_%s_epochs' %(subject,task,analysis,method,fmax))
    np.savez_compressed(f, ltcs=ltcs, times=times, labels=np.array([l.name for l in labels]))
    del ltcs
    
print 'Done.'

### Reassemble source localized epochs

In [ ]:
import os
import numpy as np
from mne.filter import low_pass_filter
from mne.time_frequency import single_trial_power
from pandas import read_csv

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Data parameters.
subjects = ['BRTU','CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
method = 'dSPM'
h_freq = 50

## Label parameters.
rois = ['dacc-lh', 'dacc-rh', 'dmpfc-lh', 'dmpfc-rh', 'dlpfc_1-lh', 'dlpfc_1-rh', 'dlpfc_2-lh', 'dlpfc_2-rh', 
        'dlpfc_3-lh', 'dlpfc_3-rh', 'dlpfc_4-lh', 'dlpfc_4-rh', 'dlpfc_5-lh', 'dlpfc_5-rh', 
        'dlpfc_6-lh', 'dlpfc_6-rh', 'pcc-lh', 'pcc-rh', 'racc-lh', 'racc-rh']

## Processing parameters.
fmax = 15
sfreq = 1450.
decim = 3
n_jobs = 3

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/source'
df = read_csv(os.path.join(root_dir, 'afMSIT_source_info.csv'))

for analysis in ['stim', 'resp']:

    for roi in rois:

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Iteratively load and merge.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        ltcs = []
        for subject in subjects:

            ## Load NPZ.
            npz = np.load(os.path.join(root_dir, 'stcs', '%s_msit_%s_%s_%s_epochs.npz' %(subject,analysis,method,h_freq)))

            ## Get label index and extract.
            ix = npz['labels'].tolist().index(roi)
            arr = npz['ltcs'][:,ix,:]

            ## Append.
            ltcs.append(arr)

        ## Concatenate.
        ltcs = np.concatenate(ltcs, axis=0)
        times = npz['times']

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Time-domain processing.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        ## Make ERP objects.
        ERPs = low_pass_filter(ltcs, sfreq, fmax, filter_length='20s', n_jobs=n_jobs)

        ## Crop times.
        if analysis == 'stim': mask = (times >= -0.5) & (times <= 2.0)
        elif analysis == 'resp': mask = (times >= -1.0) & (times <= 1.0)
        ERPs = ERPs[:, mask]
        times = times[mask]
        
        ## Save.
        np.savez_compressed(os.path.join(root_dir, 'afMSIT_source_%s_%s_%s' %(analysis,roi,fmax)), 
                            data=ERPs[:,::decim], times=times[::decim], method=method)
        
print 'Done.'

### Compute power of source localized epochs

In [ ]:
import os
import numpy as np
from mne.filter import low_pass_filter
from mne.time_frequency import single_trial_power
from pandas import read_csv

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Data parameters.
subjects = ['BRTU','CHDR', 'CRDA', 'JADE', 'JASE', 'M5', 'MEWA', 'S2']
method = 'dSPM'
h_freq = 50

## Label parameters.
rois = ['dacc-lh', 'dacc-rh', 'dmpfc-lh', 'dmpfc-rh', 'dlpfc_1-lh', 'dlpfc_1-rh', 'dlpfc_2-lh', 'dlpfc_2-rh', 
        'dlpfc_3-lh', 'dlpfc_3-rh', 'dlpfc_4-lh', 'dlpfc_4-rh', 'dlpfc_5-lh', 'dlpfc_5-rh', 
        'dlpfc_6-lh', 'dlpfc_6-rh', 'pcc-lh', 'pcc-rh', 'racc-lh', 'racc-rh']

## Time-frequency parameters.
fdict = dict(theta = (4,8), alpha = (8,15), beta = (15,30))
freqs = np.logspace( np.log10(2), np.log10(50), num=25)
n_cycles = 3
baseline = (-0.5, -0.1)
Fs = 1450.
decim = 14
n_jobs = 3

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

root_dir = '/space/sophia/2/users/EMOTE-DBS/afMSIT/source'
df = read_csv(os.path.join(root_dir, 'afMSIT_source_info.csv'))

for roi in rois:

    for analysis in ['stim', 'resp']:

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Iteratively load and merge.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        ltcs = []
        for subject in subjects:

            ## Load NPZ.
            npz = np.load(os.path.join(root_dir, 'stcs', '%s_msit_%s_%s_%s_epochs.npz' %(subject,analysis,method,h_freq)))

            ## Get label index and extract.
            ix = npz['labels'].tolist().index(roi)
            arr = npz['ltcs'][:,ix,:]

            ## Append.
            ltcs.append(arr)

        ## Concatenate.
        ltcs = np.concatenate(ltcs, axis=0)
        times = npz['times']

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Phase-lock removal.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
       
        for subject in df.Subject.unique():
            for dbs in [0, 1]:
                for cond in [0, 1]:
                    ix, = np.where((df.Subject==subject)&(df.DBS==dbs)&(df.Interference==cond))
                    ltcs[ix] -= ltcs[ix].mean(axis=0)
                    
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Compute power.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
                    
        print 'Computing single trial power: %s %s.' %(roi, analysis)
        ltcs = np.expand_dims(ltcs,1)
        power = single_trial_power(ltcs, sfreq=Fs, frequencies=freqs, n_cycles=n_cycles, 
                                    baseline=None, n_jobs=n_jobs, verbose=False)
        power = power.squeeze()
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Within-trial normalization.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        power = (power.T / np.median(power, axis=-1).T).T # median, not mean
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Prepare baseline normalization (within subject, DBS).
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
        ## Compute baseline normalization.
        if analysis == 'stim':

            ## Make time mask.
            mask = (times >= baseline[0]) & (times <= baseline[1])
            
            ## Iteratively compute over subjects.
            blnorm = []
            for subject in df.Subject.unique():
                
                ## Iteratively compute over DBS conditions..
                sbl = []
                for dbs in [0,1]:

                    ix, = np.where((df.Subject==subject)&(df.DBS==dbs))
                    sbl.append( np.apply_over_axes(np.median, power[ix][:,:,mask], axes=[0,2]).squeeze() )
            
                blnorm.append(sbl)
            
            ## Merge.
            blnorm = np.array(blnorm)
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Apply baseline normalization (within subject, DBS).
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
        ## Setup index vectors.
        n_trials, n_freqs, n_times = power.shape        
        _, subj_ix = np.unique(df.Subject, return_inverse=True)
        dbs_ix = df.DBS.as_matrix()
        trial_ix = np.arange(n_trials) 
        
        ## Main loop.
        for i,j,k in zip(trial_ix,subj_ix,dbs_ix):
            
            for m in xrange(n_times):
                
                power[i,:,m] /= blnorm[j,k,:]
                
        if analysis == 'resp': del blnorm

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Final preprocessing steps.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
            
        ## Convert to decibels.
        power = np.log10(power) * 10
            
        ## Crop times.
        if analysis == 'stim': mask = (times >= -0.5) & (times <= 2.0)
        elif analysis == 'resp': mask = (times >= -1.0) & (times <= 1.0)
        power = power[:, :, mask]
        times = times[mask]
            
        ## Iteratively average, transform, and save.
        for k,v in fdict.iteritems():
            
            ## Average across frequencies.
            ix, = np.where((freqs>=v[0])&(freqs<=v[1]))
            band = power[:,ix,:].mean(axis=1)
        
            ## Save.
            f = os.path.join(root_dir, 'afMSIT_source_%s_%s_%s' %(analysis, roi, k))
            np.savez_compressed(f, data=band[:,::decim], times=times[::decim], freqs=freqs, n_cycles=n_cycles)
        
        del power
        
print 'Done.'

### Perform permutatations
Please see: /space/sophia/2/users/EMOTE-DBS/afMSIT/scripts/cmdline/afMSIT_permutations.py

### Identify clusters, perform FDR corrections, and assemble results.

In [ ]:
import os
import numpy as np
from collections import defaultdict
from mne.stats import fdr_correction
from pandas import DataFrame, concat
from scipy.ndimage import measurements
from scipy.stats import norm

def largest_cluster(arr, threshold):
    masked = np.abs( arr ) > threshold
    clusters, n_clusters = measurements.label(masked)
    cluster_sums = measurements.sum(arr, clusters, index=np.arange(n_clusters)+1)
    if not len(cluster_sums): return 0
    else: return np.abs(cluster_sums).max()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Specify parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## File parameters.
labels = ['dacc-lh', 'dacc-rh', 'dmpfc-lh', 'dmpfc-rh', 'dlpfc_1-lh', 'dlpfc_1-rh', 'dlpfc_2-lh', 'dlpfc_2-rh', 
          'dlpfc_3-lh', 'dlpfc_3-rh', 'dlpfc_4-lh', 'dlpfc_4-rh', 'dlpfc_5-lh', 'dlpfc_5-rh', 
          'dlpfc_6-lh', 'dlpfc_6-rh', 'pcc-lh', 'pcc-rh', 'racc-lh', 'racc-rh']

analysis = 'resp'
model_name = 'revised'
freqs = ['theta', 'alpha', 'beta']
domain = ['timedomain', 'frequency'][1]

## Statistics parameters.
alpha = 0.05
min_cluster = 0.05 # seconds

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Initial preparations.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define contrasts.
if model_name == 'revised': cols = ['Intercept', 'DBS', 'Interference', 'DBSxInt', 'nsArousal', 'nsValence', 'Trial']

## Define threshold.
threshold = norm.ppf(1 - alpha/2.)

## Read in seeds.
f = '/space/sophia/2/users/EMOTE-DBS/afMSIT/scripts/cmdline/seeds.txt'
with open(f, 'r') as f: seeds = [s.strip() for s in f.readlines()]
    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main Loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#    

df = []
    
for label in labels:
    
    for freq in freqs:
    
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Load files. Assemble permutations.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        root_dir = '/autofs/space/sophia_002/users/EMOTE-DBS/afMSIT/source'
        results_dir = os.path.join(root_dir, model_name)
        out_dir = os.path.join(root_dir, 'results')

        ## Load true statistics.
        npz = np.load(os.path.join(results_dir, 'afMSIT_source_%s_%s_%s_obs.npz' %(analysis, label, freq)))
        t_scores = npz['t_scores'].squeeze()
        times = npz['times'].squeeze()

        ## Load null statistics.
        for n, seed in enumerate(seeds):
            npz = np.load(os.path.join(results_dir, 'afMSIT_source_%s_%s_%s_%s.npz' %(analysis, label, freq, seed)))
            if not n: permuted = npz['t_scores']
            else: permuted = np.concatenate([permuted, npz['t_scores']], axis=0)
                
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        ### Compute cluster statistics.
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

        ## Get info.
        n_shuffles, n_eff, n_times  = permuted.shape

        ## Iteratively compute clusters.
        results = defaultdict(list)

        for n, con in enumerate(cols):

            ## Find real clusters.
            masked = np.abs( t_scores[n] ) > threshold
            clusters, n_clusters = measurements.label(masked)
            cluster_sums = measurements.sum(t_scores[n], clusters, index=np.arange(n_clusters)+1)

            ## Compute null cluster sums.
            null_sums = np.array([largest_cluster(permuted[m, n, :], threshold) for m in xrange(n_shuffles)])

            ## Compute cluster bounds.
            tmin = np.array([times[clusters==i].min() for i in np.arange(n_clusters)+1])
            tmax = np.array([times[clusters==i].max() for i in np.arange(n_clusters)+1])

            ## Find proportion of clusters that are larger.
            p_values = [((np.abs(cs) < null_sums).sum() + 1.) / (null_sums.shape[0] + 1.) for cs in cluster_sums]
            
            ## Store results.
            for t1, t2, cs, pval in zip(tmin,tmax,cluster_sums,p_values):
                results['Freq'] += [freq]
                results['Label'] += [label]
                results['Contrast'] += [con]
                results['Tmin'] += [t1]
                results['Tmax'] += [t2]
                results['Score'] += [cs]
                results['Pval'] += [pval]

        ## Organize results and append.
        results = DataFrame(results)
        results['Tdiff'] = results['Tmax'] - results['Tmin']
        results = results[results['Tdiff']>=min_cluster]
        df.append(results)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Compute cluster statistics.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Merge dataframes.
df = concat(df)

## Organize columns and sort.
cols = ['Contrast','Label','Freq','Tmin','Tmax','Tdiff','Score','Pval']
df = df[cols].sort_values(['Contrast','Tmin']).reset_index(drop=True)

## FDR correct within contrasts.
df['FDR'] = 0
for contrast in df.Contrast.unique():
    _, fdr = fdr_correction(df.loc[df.Contrast==contrast,'Pval'], alpha)
    df.loc[df.Contrast==contrast,'FDR'] = fdr
    
## Save.
f = os.path.join(out_dir, '%s_%s_%s_results.csv' %(model_name, analysis,domain))
df.to_csv(f, index=False)
        
print 'Done.'